# Part 1 Prompt

Ok, this is a tree problem.  The language used here is about "directly orbits" but it seems that this is a parent/child branching relationship.  Then you have to traverse the tree in a kind of weird way, starting from each node and traversing up to the root.

I think a tree problem calls for classes

In [21]:
class Node:
    def __init__(self, name):
        self.name = name
        self.children = []
        
    def append_child(self, child_node):
        self.children.append(child_node)
        
    def __repr__(self):
        return f"< {self.name} >"
        

In [22]:
all_nodes = []

In [23]:
example_input = """COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L"""

In [24]:
example_input

'COM)B\nB)C\nC)D\nD)E\nE)F\nB)G\nG)H\nD)I\nE)J\nJ)K\nK)L'

In [25]:
connections = example_input.split("\n")
connections

['COM)B', 'B)C', 'C)D', 'D)E', 'E)F', 'B)G', 'G)H', 'D)I', 'E)J', 'J)K', 'K)L']

In [26]:
for connection in connections:
    parent_name, child_name = connection.split(")")
    
    # assume that the necessary nodes have not been created yet
    parent_node_in_list = False
    child_node_in_list = False
    
    # check through the list to see if they already exist
    for node in all_nodes:
        if node.name == parent_name:
            parent_node = node
            parent_node_in_list = True
            break
        elif node.name == child_name:
            child_node = node
            child_node_in_list = True
            break
    
    # if it wasn't in the list, create it and add it to the list
    if not parent_node_in_list:
        parent_node = Node(parent_name)
        all_nodes.append(parent_node)
    if not child_node_in_list:
        child_node = Node(child_name)
        all_nodes.append(child_node)
    
    parent_node.append_child(child_node)

In [27]:
all_nodes

[< COM >,
 < B >,
 < C >,
 < D >,
 < E >,
 < F >,
 < G >,
 < H >,
 < I >,
 < J >,
 < K >,
 < L >]

Ok I'm going to override `__repr__`

That looks better

In [28]:
all_nodes[0]

< COM >

In [29]:
all_nodes[0].children

[< B >]

In [30]:
all_nodes[0].children[0]

< B >

In [31]:
all_nodes[0].children[0].children

[< C >, < G >]

Ok, the data structure looks good.  Now to traverse it.  No reason to do a breadth-first traversal, is there?  So I think depth-first traversal, which means I need a recursive function

Let's do an example with a simpler tree, I can't get this to work just in my head

 - COM: orbits nothing
 - B: orbits 1
 - G: orbits 2 (via B)
 - H: orbits 3 (via G, B)
 - C: orbits 2 (via B)

So, each node adds its depth to the total

In [32]:
def traverse_tree(current_node, current_depth):
    child_depths = 0
    for child in current_node.children:
        child_depths += traverse_tree(child, current_depth + 1)
    return current_depth + child_depths

In [34]:
current_depth = 0
root_node = all_nodes[0]
total_orbits = traverse_tree(root_node, current_depth)

In [35]:
total_orbits

42

Ok, let's make sure that it can dynamically find the root

In [37]:
all_nodes

[< COM >,
 < B >,
 < C >,
 < D >,
 < E >,
 < F >,
 < G >,
 < H >,
 < I >,
 < J >,
 < K >,
 < L >]

In [38]:
center_of_mass = None
for node in all_nodes:
    if node.name == "COM":
        center_of_mass = node
        break

In [39]:
traverse_tree(center_of_mass, 0)

42

Cool cool, put everything in a function

In [70]:
def insert_or_update(parent_name, child_name, all_nodes):
    # assume that the necessary nodes have not been created yet
    parent_node_in_list = False
    child_node_in_list = False
    
    # check through the list to see if they already exist
    for node in all_nodes:
        if node.name == parent_name:
            parent_node = node
            parent_node_in_list = True
        elif node.name == child_name:
            child_node = node
            child_node_in_list = True
    
    # if it wasn't in the list, create it and add it to the list
    if not parent_node_in_list:
        parent_node = Node(parent_name)
        all_nodes.append(parent_node)
    if not child_node_in_list:
        child_node = Node(child_name)
        all_nodes.append(child_node)
    
    parent_node.append_child(child_node)

In [75]:
def build_node_list(input_str):
    connections = input_str.split("\n")
    all_nodes = []
    for connection in connections:
        parent_name, child_name = connection.split(")")
        insert_or_update(parent_name, child_name, all_nodes)
    return all_nodes

In [76]:
def count_orbits(input_str):
    all_nodes = build_node_list(input_str)
        
    center_of_mass = None
    for node in all_nodes:
        if node.name == "COM":
            center_of_mass = node
            break
            
    return traverse_tree(center_of_mass, 0), center_of_mass, all_nodes

In [43]:
count_orbits(example_input)

42

In [44]:
file_obj = open("input.txt")
full_input = file_obj.read()

In [45]:
file_obj.close()

In [47]:
full_input = full_input.strip()

In [59]:
orbits, center_of_mass, all_nodes = count_orbits(full_input)

In [60]:
orbits

1

lolwut

In [63]:
center_of_mass.children[0].children

[]

So CPF is not getting added as a child of PWZ

Now is when I want an actual debugger and I'm annoyed that I'm in a Jupyter Notebook...

In [67]:
for node in all_nodes:
    if node.name == "CPF":
        print(node)
        cpf = node

< CPF >


In [69]:
cpf.children

[< DH3 >]

In [72]:
orbits, center_of_mass, all_nodes = count_orbits(full_input)

In [73]:
orbits

271151

So, what was wrong was that I short-circuited by adding `break`s that assumed that only the parent or the child would be new information, which was true of the simple example, but not the full example.  Removing the breaks fixed it

# Part 2 Prompt

Boo this is a distance between nodes problem.

General algorithm is to find the first shared ancestor, then the distance is the distance from the shared ancestor to `YOU` plus the distance from the shared ancestor to `SAN`.  Modified slightly so it only has to go from your parent to `SAN`'s parent, maybe to stop people from just copying an algorithm they don't understand.

Boo because this is going to take some thinking as a generalized tree problem instead of a binary tree problem like it usually is.

### Finding the Shared Ancestor

Start at the root of the tree.  Baseline assumption is that the root is a shared ancestor of the two nodes, and next you have to check each of the root's children to see if they are also a shared ancestor.  It it's not a malformed tree, at most one of the root's children will be a shared ancestor of both nodes.  Keep going until no single child of the current root is a shared ancestor.

In [77]:
part_2_example = """COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN"""
all_nodes = build_node_list(part_2_example)

In [78]:
all_nodes

[< COM >,
 < B >,
 < C >,
 < D >,
 < E >,
 < F >,
 < G >,
 < H >,
 < I >,
 < J >,
 < K >,
 < L >,
 < YOU >,
 < SAN >]

In [80]:
# here we know this, in the future we will have to loop through and find it
root = all_nodes[0]
you_name = "K"
santa_name = "I"

In [84]:
def is_ancestor(current_node, search_name):
    for node in current_node.children:
        if node.name == search_name:
            return True
        elif is_ancestor(node, search_name):
            return True
    return False

In [85]:
is_ancestor(root, you_name)

True

In [86]:
is_ancestor(root, "hello, world")

False

In [87]:
def find_lowest_common_ancestor(current_node, you_name, santa_name):
    for node in current_node.children:
        if is_ancestor(node, you_name) and is_ancestor(node, santa_name):
            return find_lowest_common_ancestor(node, you_name, santa_name)
    
    return current_node

In [88]:
find_lowest_common_ancestor(root, you_name, santa_name)

< D >

Ok, looks like we are successfully finding the lowest common ancestor.  Now we have the option to start from that ancestor and traverse downwards in both directions, but a faster/better implementation would be to re-write `is_ancestor` so it tells you the distance and returns -1 if the search name was not found.  So then you only have to traverse down to both `YOU` and `SAN` once

In [89]:
def orbital_distance_if_exists(current_node, search_name, current_depth):
    for node in current_node.children:
        if node.name == search_name:
            return current_depth
        distance_if_exists = orbital_distance_if_exists(node, search_name, current_depth + 1)
        if distance_if_exists != -1:
            return distance_if_exists
    return -1

In [91]:
orbital_distance_if_exists(root, you_name, 1)

6

In [92]:
orbital_distance_if_exists(root, santa_name, 1)

4

In [93]:
def find_ancestor_and_distances(current_node, you_name, santa_name, prev_you_distance, prev_santa_distance):
    for node in current_node.children:
        santa_distance = orbital_distance_if_exists(node, santa_name, 1)
        you_distance = orbital_distance_if_exists(node, you_name, 1)
        if (santa_distance != -1) and (you_distance != -1):
            return find_ancestor_and_distances(node, you_name, santa_name, you_distance, santa_distance)
        
    return current_node, prev_you_distance, prev_santa_distance

In [94]:
find_ancestor_and_distances(root, you_name, santa_name, -1, -1)

(< D >, 3, 1)

That worked, answer is 3 + 1 = 4

Last part is conditionally finding the root, you_name, and santa_name instead of hard-coding them

In [95]:
root = None
you_name = None
santa_name = None

In [96]:
for node in all_nodes:
    if node.name == "COM":
        root = node
    
    for child in node.children:
        if child.name == "YOU":
            you_name = node.name
        elif child.name == "SAN":
            santa_name = node.name

In [97]:
root

< COM >

In [98]:
you_name

'K'

In [99]:
santa_name

'I'

Ok let's run it all

In [100]:
all_nodes = build_node_list(full_input)

In [101]:
root = None
you_name = None
santa_name = None

In [102]:
for node in all_nodes:
    if node.name == "COM":
        root = node
    
    for child in node.children:
        if child.name == "YOU":
            you_name = node.name
        elif child.name == "SAN":
            santa_name = node.name

In [103]:
root

< COM >

In [104]:
you_name

'JN3'

In [105]:
santa_name

'T1T'

In [106]:
find_ancestor_and_distances(root, you_name, santa_name, -1, -1)

(< 6C4 >, 169, 219)

Well, that is way to many hops to manually check...let's hope it works

In [107]:
169 + 219

388

Yep, that's right!